In [1]:
### User filled data here

out_dir = "/home/mwjacobs/test_stats/plots/"
save_name = "P3_combined"

# Total projections for each region should be entered here
projections = {
    'RSP': 144,
    'PM': 333,
    'AM': 669,
    'A': 319,
    'RL': 2235,
    'AL': 946,
    'LM': 2102
}

# Observed cells should be entered here
observed_cells = 4910
print(f"Observed Cells: {observed_cells}")

###

# Calculate total projections
total_projections = sum(projections.values())
print(f"Total Projections: {total_projections}")

# Imports
import sympy
from sympy import symbols, Product, Sum, Array, N
from scipy.stats import binomtest
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sympy.printing import latex

# Define symbols
N0, k = symbols('N_0 k')

# Define data as an array based on projections
s = Array(list(projections.values()))

# Perform symbolic calculations
m = len(projections) - 1
pi = (1 - Product((1 - (s[k]/N0)), (k, 0, m)).doit())
soln = sympy.solve(pi * N0 - observed_cells)
roots = [N(x).as_real_imag()[0] for x in soln]  # Use only the real parts of the roots
#roots is a numerical solution for the variable 𝑁0, which is a scaling factor related to the total number of observed cells.
print("Roots:", roots)

# Simplify the symbolic product expression
simplified_pi = sympy.simplify((1 - Product((1 - (s[k]/N0)), (k, 0, m)))).doit()
print("Simplified Pi:", simplified_pi)

# Generate and save LaTeX visualization for simplified_pi
latex_output = latex(simplified_pi)
plt.figure(figsize=(10, 6))
plt.text(0.05, 0.5, f"{latex_output}", fontsize=14, wrap=True)
plt.axis('off')
plt.title("Simplified Pi Visualization", fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(out_dir, f"{save_name}_Simplified_Pi.png"), bbox_inches='tight')
plt.close()

# Calculate probabilities from array values
ps = [(x / total_projections) for x in list(projections.values())]
print("Probabilities:", ps)

# Define a dictionary for region-specific probabilities dynamically
psdict = {region: prob for region, prob in zip(projections.keys(), ps)}
print("Region-specific Probabilities:", psdict)

# Solve for a specific probability
pe = symbols('p_e')
solution = sympy.solve((1 - (1 - pe)**len(projections)) * total_projections - observed_cells, [pe], force=True)
print("Solution for p_e:", solution)

# Dynamically derive pe_num from solution
# If there are multiple solutions, modify this to select the desired one (e.g., first positive root)
if solution:
    pe_num = float(solution[0])  # Select the first solution by default
    print("Derived pe_num:", pe_num)
else:
    raise ValueError("No valid solution for pe found. Ensure inputs are correct.")

# Dynamically derive the hardcoded value 0.007623338558498366
# Using probabilities as a base for estimation
scaled_value = psdict['RSP'] * psdict['PM']

# Numerical calculations with the dynamically derived value of p_e
calculated_value = (1 - (1 - pe_num)**len(projections)) * total_projections
print("Calculated Value:", calculated_value)

# Generate and save LaTeX visualization for calculated_value
latex_output_value = latex(calculated_value)
plt.figure(figsize=(10, 6))
plt.text(0.05, 0.5, f"{latex_output_value}", fontsize=14, wrap=True)
plt.axis('off')
plt.title("Calculated Value Visualization", fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(out_dir, f"{save_name}_Calculated_Value.png"), bbox_inches='tight')
plt.close()

# Further computations
product_term = (pe_num)**2 * (1 - pe_num)**(len(projections) - 2)
print("Product Term:", product_term)

weighted_value = scaled_value * total_projections
print("Weighted Value:", weighted_value)

std_dev = np.sqrt(scaled_value * total_projections * (1 - scaled_value))
print("Standard Deviation:", std_dev)

# Perform a binomial test
binomial_test_result = binomtest(94, n=total_projections, p=scaled_value).pvalue
print("Binomial Test Result (P-Value):", binomial_test_result)

# Output results to CSV and graphics
output_data = {
    'Roots': roots,
    'Simplified Pi': [simplified_pi],
    'Probabilities': ps,
    'Region-specific Probabilities': list(psdict.values()),
    'Solution for p_e': solution,
    'Derived pe_num': [pe_num],
    'Calculated Value': [calculated_value],
    'Product Term': [product_term],
    'Weighted Value': [weighted_value],
    'Standard Deviation': [std_dev],
    'Binomial Test Result (P-Value)': [binomial_test_result]
}

# Ensure output directory exists
os.makedirs(out_dir, exist_ok=True)

# Save each result as a CSV
for key, value in output_data.items():
    output_file = os.path.join(out_dir, f"{save_name}_{key.replace(' ', '_')}.csv")
    pd.DataFrame({key: value}).to_csv(output_file, index=False)

# Generate and save graphics for selected results
plt.figure(figsize=(8, 5))
plt.bar(psdict.keys(), psdict.values())
plt.title("Region-specific Probabilities")
plt.ylabel("Probability")
plt.xlabel("Region")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(out_dir, f"{save_name}_Region_Probabilities.png"))
plt.close()

# Example for roots (scatterplot)
plt.figure(figsize=(8, 5))
plt.scatter(range(len(roots)), roots)  # Use the real parts of the roots
plt.title("Roots")
plt.ylabel("Root Value")
plt.xlabel("Index")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(out_dir, f"{save_name}_Roots.png"))
plt.close()


Observed Cells: 4910
Total Projections: 6748
Roots: [143.284332833202, 8067.74431336253, 253.540577308847, 253.540577308847, 339.804185556290, 339.804185556290]
Simplified Pi: 1 - (N_0 - 2235)*(N_0 - 2102)*(N_0 - 946)*(N_0 - 669)*(N_0 - 333)*(N_0 - 319)*(N_0 - 144)/N_0**7
Probabilities: [0.02133965619442798, 0.0493479549496147, 0.09914048606994665, 0.0472732661529342, 0.3312092471843509, 0.14018968583283936, 0.3114997036158862]
Region-specific Probabilities: {'RSP': 0.02133965619442798, 'PM': 0.0493479549496147, 'AM': 0.09914048606994665, 'A': 0.0472732661529342, 'RL': 0.3312092471843509, 'AL': 0.14018968583283936, 'LM': 0.3114997036158862}
Solution for p_e: [CRootOf(3374*x**7 - 23618*x**6 + 70854*x**5 - 118090*x**4 + 118090*x**3 - 70854*x**2 + 23618*x - 2455, 0), CRootOf(3374*x**7 - 23618*x**6 + 70854*x**5 - 118090*x**4 + 118090*x**3 - 70854*x**2 + 23618*x - 2455, 1), CRootOf(3374*x**7 - 23618*x**6 + 70854*x**5 - 118090*x**4 + 118090*x**3 - 70854*x**2 + 23618*x - 2455, 2), CRootOf(337